In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from formatting_data import get_data_frame

In [2]:
df, game_df = get_data_frame()

In [4]:
game_df.head()

,game_id,game_name
0,0,Counter-Strike
1,1,Team Fortress Classic
2,2,Day of Defeat
3,3,Deathmatch Classic
4,4,Half-Life: Opposing Force


In [1]:
class Collaborative_filtering:
    def __init__(self):
        print('Model initialised')

    def load_data(self, df, game_df):
        self.df = df
        self.game_df = game_df
        self.utility_matrix = df.pivot_table(index='user_id', columns='game_id', values='playtime')
        print("Data loaded sucessfully")

    def train(self, X_train, y_train):
        print('Training model')

    def predict(self, X):
        print('Predicted')

In [2]:
model = Collaborative_filtering


In [17]:
utility_matrix = df.pivot_table(index='user_id', columns='game_id', values='playtime', fill_value=0)

In [22]:
utility_matrix.mean(axis=1)

user_id
0        0.455176
1        0.286004
2        0.549792
3        0.503563
4        0.461894
           ...   
70907    0.175102
70908    0.500448
70909    0.357984
70910    0.393197
70911    0.322285
Length: 70912, dtype: float64

In [56]:
utility_matrix.iloc[0]

game_id
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
10973   NaN
10974   NaN
10975   NaN
10976   NaN
10977   NaN
Name: 0, Length: 10978, dtype: float64